<a href="https://colab.research.google.com/github/xCosmicx/ATA/blob/main/week12/bert-finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT for Text Classification

One of the approaches where we can use BERT for downstream task such as text classification is to do fine-tuning of the pretrained model. 

In this lab, we will see how we can use a pretrained DistilBert Model and fine-tune it with custom training data for text classification task. 

At the end of this session, you will be able to:
- prepare data and use model-specific Tokenizer to format data suitable for use by the model
- configure the transformer model for fine-tuning 
- train the model for binary and multi-class text classification


### Install Hugging Face Transformers library

If you are running this notebook in Google Colab, you will need to install the Hugging Face transformers library as it is not part of the standard environment.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 15.5 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
     |████████████████████████████████| 101 kB 14.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

## Data Preparation

In [3]:
test_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_test.csv'
train_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_train.csv'

In [4]:
train_df = pd.read_csv(train_data_url)
test_df = pd.read_csv(test_data_url)

In [5]:
train_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


The train set has 40000 samples. We will use only a small subset (e.g. 2000) samples for finetuning our pretrained model. Similarly we will use a smaller test set for evaluating our model.  We use dataframe's `sample()` to randomly select a subset of samples.

In [6]:
TRAIN_SIZE = 2000
TEST_SIZE = 200 

train_df = train_df.sample(n=TRAIN_SIZE)
test_df = test_df.sample(n=TEST_SIZE)

We now convert the text label into numeric values of 0 (negative) and 1 (positive) 

In [7]:
train_df['sentiment'] =  train_df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
test_df['sentiment'] =  test_df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

In [8]:
train_df.sentiment.value_counts()

0    1022
1     978
Name: sentiment, dtype: int64

In [9]:
train_texts = train_df['review']
train_labels = train_df['sentiment']
test_texts = test_df['review']
test_labels = test_df['sentiment']

In [10]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

## Tokenization

We will now load the DistilBert tokenizer for the pretrained model "distillbert-base-uncased".  The tokenizer helps to produce the input tokens that are suitable to be used by the DistilBert model, e.g. it automatically append the \[CLS\] token in the front of the sequence of tokens and the \[SEP\] token at the end of the sequence of tokens , and also the attention mask for those padded positions in the input sequence of tokens.

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

The DistilBERT tokenizer (identical to Bert tokenizer) use WordPiece vocabulary. It has close to 30000 words. Each word has its own ids, we would need to map the tokens to those ids.

In [12]:
print(f"Tokenizer vocab size = {tokenizer.vocab_size}")
print(list(tokenizer.vocab.keys())[6000:6020])

Tokenizer vocab size = 30522
['fowler', 'brightness', '##cured', 'effects', 'δ', 'transportation', '##ij', 'published', 'investigator', 'urine', 'resignation', 'hiking', '##peed', '##tidae', 'lai', '36', 'k', '##ously', 'ɫ', 'edward']


Let us take a closer look at the output of the tokenization process. 

We notice that the tokenizer will return a dictionary of two items 'input_ids' and 'attention_mask'. The input_ids contains the IDs of the tokens. While the 'attention_mask' contains the masking pattern for those padded positions. If you are using BERT tokenizer, there will be additional item called 'token_type_ids'.

We also notice that for the example sentence, the word 'Transformer' is being broken up into two tokens 'Trans' and '##former'. The '##' means that the rest of the token should be attached to the previous one.

We also see that the tokenizer appended \[CLS\] to the beginning of the token sequence, and \[SEP\] at the end. 

In [13]:
test_sentence = "Transformer is really good for Natural Language Processing."

encoding = tokenizer(test_sentence, padding=True, truncation=True)
print(f"Encoding keys:  {encoding.keys()}\n")

print(f"token ids: {encoding['input_ids']}\n")
print(f"attention_mask: {encoding['attention_mask']}\n")
print(f"tokens: {tokenizer.convert_ids_to_tokens(encoding['input_ids'])}")


Encoding keys:  dict_keys(['input_ids', 'attention_mask'])

token ids: [101, 10938, 2121, 2003, 2428, 2204, 2005, 3019, 2653, 6364, 1012, 102]

attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

tokens: ['[CLS]', 'transform', '##er', 'is', 'really', 'good', 'for', 'natural', 'language', 'processing', '.', '[SEP]']


Now let's go ahead and tokenize our texts. But before we do so, we need to convert the pandas series to list first as the tokenizer cannot work with pandas series or dataframe directly. 

In [14]:
train_texts = train_texts.to_list()
train_labels = train_labels.to_list()
val_texts = val_texts.to_list()
val_labels = val_labels.to_list()
test_texts = test_texts.to_list()
test_labels = test_labels.to_list()

In [15]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, padding=True, truncation=True)

We then create a tensorflow dataset using the encodings and the labels.

In [16]:
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(batch_size)

## Fine-tuning the model

Now let us fine-tune the pre-trained model by training it with our custom dataset.  

We will instantiate a pretrained model 'distilbert-base-uncased', using `TFAutoModelForSequenceClassification`, and passing `num_labels=2` to indicate we want to train a 2-class (binary) classifier.

The model is a `tf.keras.Model` subclass. So you can train the model using Keras API such as `fit()`.

In [17]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",num_labels=2)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Transformer models benefit from a much lower learning rate than the default used by Adam, which is 1e-3. In this training, we will start the training with 5e-5 (0.00005) and slowly reduce the learning rate over the course of training. In the literature, you will sometimes see this referred to as decaying or annealing the learning rate. In Keras, the best way to do this is to use a learning rate scheduler. A good one to use is PolynomialDecay. Despite the name, with default settings it simply linearly decays the learning rate from the initial value to the final value over the course of training, which is exactly what we want. In order to use a scheduler correctly, though, we need to tell it how long training is going to be. We compute that as `num_train_steps` below.

In [18]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

num_epochs = 1

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Since our dataset is already batched, we can simply take the len.
num_train_steps = len(train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

Now we will just compile the model with the learning rate scheduler and the loss function and train our model for 1 epoch. 

Note that the transformer model output logits directly instead of going through a softmax layer. In your loss function, you will need to set `from_logits=True`.


In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

opt = Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

100/100 [==============================] - 72s 617ms/step - loss: 0.4245 - accuracy: 0.7975 - val_loss: 0.3009 - val_accuracy: 0.8825


You will notice that validation accuracy reaches around 89%.  Let's evaluate on our test set. We should see around the same accuracy. 


In [20]:
model.evaluate(test_dataset)

13/13 [==============================] - 2s 185ms/step - loss: 0.2733 - accuracy: 0.9050


[0.2732582092285156, 0.9049999713897705]

Let's just go ahead and save our model for inference later. Note that we use transformers library specific save method `save_pretrained()` instead of normal keras model save.

In [21]:
model.save_pretrained('finetuned_model')

## Try out the model

Now let's try out our model with our own sentence.  We first load our saved fined-tuned model using `from_pretrained()` method and specify the folder name where we saved the model to.

In [22]:
my_model = TFAutoModelForSequenceClassification.from_pretrained(
        "finetuned_model")

Some layers from the model checkpoint at finetuned_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at finetuned_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
text = input('Write your review here:')

Write your review here:Wow, can't believe I just wasted my money to watch this


In [24]:
inputs = tokenizer(text, return_tensors="tf")
output = my_model(inputs)
pred_prob = tf.nn.softmax(output.logits, axis=-1)
print(pred_prob)
pred = np.argmax(pred_prob)
print(pred)
if pred == 1:
    print('positive')
else:
    print('negative')

tf.Tensor([[0.66815275 0.3318473 ]], shape=(1, 2), dtype=float32)
0
negative
